In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [2]:
def prepare_data_hand_right(df):
    df['Type'] = 1
    df = df.dropna()
    df.rename(str.strip, axis='columns', inplace=True)
    add_wanted_columns(df)
#     df = df.drop(['Time', 'Frame ID', 'Hand Type', '# hands'], axis = 1)
    df = df.add_suffix(' right')
    return df

In [3]:
def combine_two_samples(df):
    hand_left = pd.DataFrame()
    hand_right = pd.DataFrame()
    for i in range(len(df)):
        if (i%2 == 0):
            hand_left = pd.concat([hand_left, df.iloc[i:i+1]])
        else:
            hand_right = pd.concat([hand_right, df.iloc[i:i+1]])

    hand_left = hand_left.add_suffix(' left')
    hand_right = hand_right.add_suffix(' right')
    hand_left.set_index(np.arange(len(hand_left)), inplace=True)
    hand_right.set_index(np.arange(len(hand_right)), inplace=True)
    result = pd.concat([hand_left, hand_right], axis=1, join='inner')
    result

In [4]:
def prepare_data(df, df_path):
    df['Type'] = sync_type(df_path)
    df = df.dropna()
    df.rename(str.strip, axis='columns', inplace=True)
    add_wanted_columns(df)
    df = df[df['Time']>=df['Time'].values[0]+7]
    total_time = df.iloc[len(df)-1,0] - df.iloc[0,0]
    number_of_jumps = int(total_time//3)
#     df = df.drop(['Time', 'Frame ID', 'Hand Type', '# hands'], axis = 1)
    i=0
    df2 = pd.DataFrame()
    if df['Type'].values[0] == 1 :
        while(i*number_of_jumps<len(df)):
            df2 = pd.concat([df2,df.iloc[i*number_of_jumps:i*number_of_jumps+1]])
            i=i+1
        df2 = df2.add_suffix(' left')
        result = pd.concat([df2, df_right_hand], axis=1, join='inner')
        return result
    
    else:
        while(i*number_of_jumps<len(df)):
            df2 = pd.concat([df2,df.iloc[i*number_of_jumps:i*number_of_jumps+2]])
            i=i+1
        return combine_two_samples(df2)
    
#     return combine_two_samples(df2)
#     return df2

In [5]:
def sync_type(path_name):
    if "Sync" in path_name:
        return 3
    if "Spontan" in path_name:
        return 2
    return 1


In [6]:
def add_wanted_columns(df):
    df['deltaX'] = df['Position X']
    df['deltaY'] = df['Position Y']
    df['deltaZ'] = df['Position Z']

    for i in range(2, len(df)):
        df['deltaX'][i] = abs(df['Position X'][i] - df['Position X'][i-2])
        df['deltaY'][i] = abs(df['Position Y'][i] - df['Position Y'][i-2])
        df['deltaZ'][i] = abs(df['Position Z'][i] - df['Position Z'][i-2])
    

In [7]:
df_right_hand = pd.read_csv('Data\HandRight.csv')
df_right_hand = prepare_data_hand_right(df_right_hand)
df_right_hand

final_training_df = pd.DataFrame()
for name in glob.glob("Data/Training/*/*"):
    df = pd.read_csv(name)
    df = prepare_data(df, name)
    final_training_df = pd.concat([final_training_df,df], ignore_index=True)

C:\Users\matan\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-4c46ad919ecc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['deltaX'] = df['Position X']
<ipython-input-6-4c46ad919ecc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [8]:
final_training_df

,Time left,Frame ID left,Hand Type left,# hands left,Position X left,Position Y left,Position Z left,Velocity X left,Velocity Y left,Velocity Z left,...,Elbow pos X right,Elbow Pos Y right,Elbow Pos Z right,Grab Strenth right,Grab Angle right,Pinch Strength right,Type right,deltaX right,deltaY right,deltaZ right
0,252.6601,118956,left,1,-117.10010,218.1561,138.06620,-53.72382,113.05610,138.402500,...,280.63840,142.54010,206.7526,0.0,0.636596,0.0,1,1.397240,10.95640,2.152277
1,253.0250,118999,left,1,-79.64585,215.0029,97.35830,325.79870,-443.03620,-356.436200,...,232.12740,152.17700,206.1120,0.0,0.155698,0.0,1,1.349670,11.10810,0.595890
2,253.3890,119041,left,1,-70.99510,131.2520,26.87347,-216.18780,268.71140,-38.333800,...,234.49400,140.98970,212.9460,0.0,0.116087,0.0,1,0.643940,5.05810,1.759128
3,253.7535,119083,left,1,-119.59160,247.2985,53.85407,-30.92409,402.77460,80.549030,...,240.91750,108.75280,207.8943,0.0,0.184295,0.0,1,0.083857,8.71950,4.588070
4,254.1185,119125,left,1,-84.25404,306.7198,45.60870,120.72030,-228.24530,-6.927217,...,249.87540,142.95530,271.2112,0.0,0.436759,0.0,1,1.555400,0.35190,1.497710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2547,761.1591,59722,left,1,-128.81230,233.8647,39.79329,-15.86829,20.60490,8.012849,...,155.56380,17.35931,310.1947,0.0,0.303011,0.0,1,3.959602,3.16210,1.856620
2548,761.8923,59779,left,1,-117.99470,228.6535,57.30731,85.41186,-165.35030,-144.948200,...,91.45721,60.79039,361.2808,0.0,0.080270,0.0,1,0.791018,0.95710,1.845250
2549,762.6262,59837,left,1,-110.57080,215.7471,32.38994,422.00760,-598.29410,-219.324200,...,10.38490,24.93950,348.1182,0.0,0.237385,0.0,1,0.823856,3.52690,0.447200
2550,763.3591,59906,left,1,-103.61690,204.1008,34.14046,-85.16750,77.61469,-17.180190,...,116.58040,98.10934,372.6849,0.0,0.540797,0.0,1,1.354180,4.13560,0.601390


In [9]:
final_training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2552 entries, 0 to 2551
Data columns (total 52 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Time left             2552 non-null   float64
 1   Frame ID left         2552 non-null   int64  
 2   Hand Type left        2552 non-null   object 
 3   # hands left          2552 non-null   int64  
 4   Position X left       2552 non-null   float64
 5   Position Y left       2552 non-null   float64
 6   Position Z left       2552 non-null   float64
 7   Velocity X left       2552 non-null   float64
 8   Velocity Y left       2552 non-null   float64
 9   Velocity Z left       2552 non-null   float64
 10  Pitch left            2552 non-null   float64
 11  Roll left             2552 non-null   float64
 12  Yaw left              2552 non-null   float64
 13  Wrist Pos X left      2552 non-null   float64
 14  Wrist Pos Y left      2552 non-null   float64
 15  Wrist Pos Z left     